In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
print('test')

test


In [1]:
import tensorflow as tf

cifar = tf.keras.datasets.cifar100
(x_train, y_train), (x_test, y_test) = cifar.load_data()
model = tf.keras.applications.ResNet50(
    include_top=True,
    weights=None,
    input_shape=(32, 32, 3),
    classes=100,)

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
model.compile(optimizer="adam", loss=loss_fn, metrics=["accuracy"])
model.fit(x_train, y_train, epochs=5, batch_size=64)

NotFoundError: dlopen(/Users/danieldreger/miniforge3/envs/dd10/lib/python3.10/site-packages/tensorflow-plugins/libmetal_plugin.dylib, 0x0006): Symbol not found: __ZN10tensorflow16TensorShapeProtoC1ERKS0_
  Referenced from: <10B7FC95-0B10-3E4E-84D0-79A2D52E4D78> /Users/danieldreger/miniforge3/envs/dd10/lib/python3.10/site-packages/tensorflow-plugins/libmetal_plugin.dylib
  Expected in:     <C104091C-297A-300E-A02F-509BCA2330E3> /Users/danieldreger/miniforge3/envs/dd10/lib/python3.10/site-packages/tensorflow/python/_pywrap_tensorflow_internal.so

In [ ]:
# import tensorflow as tf
from tensorflow.keras.models import Sequential
print('test')


In [ ]:
#  Import nba data
nba_gamestats_df = pd.read_csv("Basketball_Data/curated_game_stats.csv")
print(nba_gamestats_df.head())

nba_advancedgame_df = pd.read_csv("Basketball_Data/Advanced_Game_Stats_2001-2003.csv")
nba_advancedgame_df.head()

In [ ]:
#Check types for nba game stats
nba_gamestats_df.dtypes

In [ ]:
#Check types for nba advanced stats
nba_advancedgame_df.dtypes

In [ ]:
#Test matching dates needed for merge
nba_gamestats_df.loc[nba_gamestats_df["Date"] == nba_advancedgame_df["Date"][0]]

In [ ]:
#Drop columns not needed, rename column
nba_advancedgame_df = nba_advancedgame_df.drop(["Result", "Rk", "FTr",	'3PAr',	'eFG%.1',	'FT/FGA',	'ORtg.1',	'FTr.1',	'3PAr.1',	'TS%.1',	'eFG%.2',	'FT/FGA.1'], axis=1)

nba_advancedgame_df.rename({
  "Unnamed: 4" : "Home/Away"
}, axis=1, inplace=True)


nba_advancedgame_df.head()

In [ ]:
#Drop columns not needed
nba_gamestats_df = nba_gamestats_df.drop(["Unnamed: 0"], axis=1)

nba_gamestats_df.head()

In [ ]:
complete_stats_df = pd.merge(nba_gamestats_df, nba_advancedgame_df, on=["Date", "Team", "Opp"])

In [ ]:
complete_stats_df

In [ ]:
#Transform Dataframe
complete_stats_df=complete_stats_df.astype({
  'PTS' : 'float',
  'Opponent_Points' : 'float',
  'Home/Away': 'string',
  'FG': 'float',
  'FGA': 'float',
  '2PA': 'float',
  '3P': 'float',
  '3PA': 'float',
  'Team': 'string',
  'Opp': 'string'
})

complete_stats_df.dtypes

In [ ]:
#Add Column and convert home/away
complete_stats_df["Pts Difference"] = complete_stats_df["PTS"] - complete_stats_df["Opponent_Points"]
complete_stats_df['Home/Away'] = complete_stats_df['Home/Away'].replace(['@', ''], ['Away', 'Home'])
complete_stats_df['Home/Away'].fillna(value="Home", inplace=True)

complete_stats_df.head()

In [ ]:
complete_stats_df.drop(["Date", "Opp", "Opponent_Points", "Pts Difference", 'Team'], axis=1, inplace=True)

In [ ]:
complete_stats_df.head()

In [ ]:
# Split our preprocessed data into our features and target arrays
y = complete_stats_df['PTS']
X = complete_stats_df.drop(['PTS'], axis=1)

In [ ]:
#One Hot Encoding
X = pd.get_dummies(X, columns=["Home/Away"])

In [ ]:
X.head()

In [ ]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()


# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
#Instantiate Neural Network Model
nn_model = tf.keras.models.Sequential()

unit = 2 * X_train_scaled.shape[1]

#Dense Layers
nn_model.add(tf.keras.layers.Dense(units=unit, activation="relu", input_dim=X_train_scaled.shape[1]))
nn_model.add(tf.keras.layers.Dense(units=unit/4, activation="relu"))


#Output Layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="linear"))


In [ ]:
#Summary of Neural Network
nn_model.summary()

In [ ]:
#Compile the model
nn_model.compile(loss="mean_squared_error", optimizer="adam", metrics=["accuracy"])

#Train model
fit_model = nn_model.fit(X_train, y_train, epochs=200)

In [ ]:
#
#
#

model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")